Import libraries

In [18]:
import os
import sys
import glob
import zipfile
from zipfile import BadZipFile
import datetime
import pandas as pd
import hyp3_sdk as sdk
from hyp3_sdk import asf_search
from hyp3lib import cutGeotiffs
from tqdm.notebook import trange, tqdm
# from tqdm.auto import tqdm

Enter Hyp3 credentials

In [2]:
hyp3 = sdk.HyP3(prompt=True)
my_info = hyp3.my_info()
print(f"Remaining Quota: {my_info['quota']['remaining']}")

NASA Earthdata Login username:  forrestfwilliams
NASA Earthdata Login password:  ····················


Remaining Quota: 2256


Create download list

In [6]:
# Project name
# project_name = 'ifg_a_p81_f1048_20x4'
project_name = 'p81f1048_10x2_2'

# Set date bounds
start_date = datetime.datetime(2017, 12, 31)
end_date = datetime.datetime(2021, 12, 31)

# Read in 
df = pd.read_csv('slcs_asc_path81_frame1048.csv',usecols=['Granule Name', 'Acquisition Date'])

# Convert and limit by date
df['Acquisition Date'] = pd.to_datetime(df['Acquisition Date'], format='%Y-%m-%dT%H:%M:%S')
df = df.loc[(df['Acquisition Date']>=start_date) & (df['Acquisition Date']<=end_date)]
df = df.sort_values('Acquisition Date')

granules = list(df['Granule Name'])
print(f'Will create interferograms for {len(granules)} SLCs')

Will create interferograms for 96 SLCs


Request Ifg generation

In [9]:
insar_jobs = sdk.Batch()
first = True
for reference in tqdm(granules):
    neighbors_metadata = asf_search.get_nearest_neighbors(reference, max_neighbors=3)
    for secondary_metadata in neighbors_metadata:
        secondary = secondary_metadata['granuleName']
        if first:
            insar_jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, include_inc_map=True,
                                                include_dem=True, looks='10x2', include_wrapped_phase=False)
            first = False
        else:
            insar_jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, looks='10x2', include_wrapped_phase=False)

  0%|          | 0/96 [00:00<?, ?it/s]

KeyboardInterrupt: 

Check progress and download

In [11]:
# batch = hyp3.find_jobs(name=project_name)

# if batch.complete():
#     #filter to only succeeded jobs
#     succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)

#     #download files if not downloaded already
#     for job in succeeded_jobs.jobs:
#         filename = job.to_dict()['files'][0]['filename']
#         location = os.path.join(project_name,filename)
#         if not os.path.exists(location):
#             job.download_files(location=project_name,create=True)
        
#     # #download files if not downloaded already
#     # file_list = succeeded_jobs.download_files(location=project_name,create=True)
# else:
#     #to get updated information
#     batch = hyp3.refresh(batch)
#     #or to wait until completion and get updated information (which will take a fair bit)
#     batch = hyp3.watch(batch)

Unzip files and clip to same extent

In [16]:
os.chdir('/mnt/i/hyp3_downloads')

In [17]:
#filter to only succeeded jobs
succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)

#download files if not downloaded already
for job in succeeded_jobs.jobs:
    filename = job.to_dict()['files'][0]['filename']
    location = os.path.join(project_name,filename)
    if not os.path.exists(location):
        job.download_files(location=project_name,create=True)

S1BB_20210218T070641_20210326T070641_VVP036_INT40_G_ueF_10FF.zip:   0%|          | 0/377608017 [00:00<?, ?it/s…

S1BB_20210302T070641_20210326T070641_VVP024_INT40_G_ueF_79BB.zip:   0%|          | 0/377145576 [00:00<?, ?it/s…

S1BB_20210314T070641_20210326T070641_VVP012_INT40_G_ueF_3BF5.zip:   0%|          | 0/371860434 [00:00<?, ?it/s…

S1BB_20210206T070642_20210314T070641_VVP036_INT40_G_ueF_88DA.zip:   0%|          | 0/379786584 [00:00<?, ?it/s…

S1BB_20210218T070641_20210314T070641_VVP024_INT40_G_ueF_E3E7.zip:   0%|          | 0/382440502 [00:00<?, ?it/s…

...

S1BB_20180105T070624_20180117T070623_VVP012_INT40_G_ueF_D024.zip:   0%|          | 0/384229908 [00:00<?, ?it/s…

S1BB_20171224T070624_20180117T070623_VVP024_INT40_G_ueF_E6B9.zip:   0%|          | 0/375552185 [00:00<?, ?it/s…

S1BB_20171130T070625_20180105T070624_VVP036_INT40_G_ueF_F516.zip:   0%|          | 0/375725464 [00:00<?, ?it/s…

S1BB_20171212T070625_20180105T070624_VVP024_INT40_G_ueF_8AC1.zip:   0%|          | 0/381566792 [00:00<?, ?it/s…

S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B.zip:   0%|          | 0/724966736 [00:00<?, ?it/s…

In [22]:
# project_name = 'tongariro_asc'

zips = glob.glob(os.path.join(project_name,'*.zip'))
unws = [[x,f'{os.path.join(x.split(".")[0],os.path.basename(x).split(".")[0])}_unw_phase.tif'] for x in zips]

zips = [x[0] for x in unws if not os.path.exists(x[1])]
folders = [x.split('.')[0] for x in zips]

bad_files=[]

for zip_file, folder in tqdm(zip(zips, folders), total=len(zips)):
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(project_name)
    except BadZipFile:
        bad_files.append(os.path.basename(zip_file))

if len(bad_files) > 0:
    print(f'These files were invalid zips:\n{bad_files}')

  0%|          | 0/278 [00:00<?, ?it/s]

In [23]:
folders = glob.glob(os.path.join(project_name,'S1*'))
all_images = []
for suffix in ['dem','inc_map','water_map','unw_phase','corr']:
    all_images += [os.path.join(x,f'{os.path.basename(x)}_{suffix}.tif') for x in folders]

exists = [x for x in all_images if os.path.exists(x)]
to_clip = [x for x in exists if not os.path.exists(f'{x[:-4]}_clip.tif')]

cutGeotiffs.cutFiles(to_clip)

Clipping coordinates: (266020.0, 5753820.0, 548860.0, 5551620.0)
Found overlap size of 7071x5055
Pixsize : x = 40.0 y = -40.0
    clipping file p81f1048_10x2_2/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B_dem.tif to create file p81f1048_10x2_2/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B_dem_clip.tif
    clipping file p81f1048_10x2_2/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B_inc_map.tif to create file p81f1048_10x2_2/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B/S1BB_20171224T070624_20180105T070624_VVP012_INT40_G_ueF_0A8B_inc_map_clip.tif
    clipping file p81f1048_10x2_2/S1BB_20210218T070641_20210326T070641_VVP036_INT40_G_ueF_10FF/S1BB_20210218T070641_20210326T070641_VVP036_INT40_G_ueF_10FF_unw_phase.tif to create file p81f1048_10x2_2/S1BB_

Done!